In [1]:
import Beer
from BeerScraper import create_data
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from bs4 import BeautifulSoup
import requests
import re
import nltk

pd.set_option('max_colwidth', 300)

df=pd.read_csv('beer.csv', header='infer')
df.head()

,name,brewery,style,rating,review
0,Big Rock Ale,Big Rock Brewery,Scottish Ale,3.90,"smell  soft hop aroma with significant malt scents. this one smells very creamy. taste  and creamy it is. the traditional irish flavors come out at the tongue. this is creamy, not like a cream ale, but close. the malt is big, buttery, and very smooth. the hops are very unique. this is not a sh..."
1,Flip Ale,Dogfish Head Craft Brewery,Old Ale,4.08,on tap at dfh rehoboth... collab with eatily... cardamom and red wine must. golden orange. .no head. typical dfh yeast aroma. ..some spice and maybe a belgian influence. sweet spicy and somewhat fruity.. not much old ale characteristic. too light for that. its still tasty. the cardamom does add...
2,The Almond Marzen Project - Beer Camp #26,Sierra Nevada Brewing Co.,Märzen / Oktoberfest,3.78,"nice auburn impressions, tons of clarity, solid inch of off white head. aroma was a little bit sweet and nutty. taste gave a little more sweetness, stayed away from hops and bitterness, relatively light bodied. nothing almond came out of it that was obvious. kind of a fancied up oktoberfest ..."
3,Perpetual Burn IPA,Rock Bottom Restaurant & Brewery,American IPA,3.88,"s - piney, a little cat piss in a good way, skunky bud. t - some caramel front, tons of hops at the finish. m - medium body, a little too sharp and unbalanced for me. d - very sessionable considering the hops and abv."
4,Hop Monster DRIPA,Iron Hill Brewery & Restaurant,American Double / Imperial IPA,4.14,"double red ipa, full-bodied rich caramel malt character with a generous amount of american hops. beware the monster! a: this was a clear, amber tinted copper color. it was served to me with a firm, frothy cap. s: sugar cookie/short bread malts with a little honey and a very strong blast of tro..."


In [10]:
print(df['review'][0])

smell  soft hop aroma with significant malt scents. this one smells very creamy. taste  and creamy it is. the traditional irish flavors come out at the tongue. this is creamy, not like a cream ale, but close. the malt is big, buttery, and very smooth. the hops are very unique. this is not a sharp hop flavor. this is an easy, saturated, well-mixed blend that plays a complimenting second fiddle to the malt base. no sweetness. the finish is nutty and big. this ale changes personalities at the end. mouthfeel  lightly carbonated and exceptionally smooth and creamy. drinkability  very creamy. i know that im repeating myself, but this ale is creamy like peanut butter. its actually kind of nutty to boot. creamy. 


In [40]:
def remove_punc(word):
    # Removes punctuation from a word
    return re.sub(r'[^\w\s]', '', word)

for i in range(len(df)):
    # split into list of words
    words = df['review'][i].split()  
    # Remove stopwords
    words = [word for word in words if word not in nltk.corpus.stopwords.words("english")]
    # Remove puncutation
    words = [remove_punc(word) for word in words]
    # Remove numbers
    words = [word for word in words if not word.isdigit()]
    # Remove some particular words that break things
    #words = [word for word in words if word not in ['name', 'style', 'brewery', 'rating', '']]
    df['words'][i] = words



/Users/brentmarijensen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [46]:
df.head()

,name,brewery,style,rating,review,words
0,Big Rock Ale,Big Rock Brewery,Scottish Ale,3.90,"smell  soft hop aroma with significant malt scents. this one smells very creamy. taste  and creamy it is. the traditional irish flavors come out at the tongue. this is creamy, not like a cream ale, but close. the malt is big, buttery, and very smooth. the hops are very unique. this is not a sh...","[smell, , soft, hop, aroma, significant, malt, scents, one, smells, creamy, taste, , creamy, is, traditional, irish, flavors, come, tongue, creamy, like, cream, ale, close, malt, big, buttery, smooth, hops, unique, sharp, hop, flavor, easy, saturated, wellmixed, blend, plays, complimenting, seco..."
1,Flip Ale,Dogfish Head Craft Brewery,Old Ale,4.08,on tap at dfh rehoboth... collab with eatily... cardamom and red wine must. golden orange. .no head. typical dfh yeast aroma. ..some spice and maybe a belgian influence. sweet spicy and somewhat fruity.. not much old ale characteristic. too light for that. its still tasty. the cardamom does add...,"[tap, dfh, rehoboth, collab, eatily, cardamom, red, wine, must, golden, orange, no, head, typical, dfh, yeast, aroma, some, spice, maybe, belgian, influence, sweet, spicy, somewhat, fruity, much, old, ale, characteristic, light, that, still, tasty, cardamom, add, nice, flavor, still, kind, lacki..."
2,The Almond Marzen Project - Beer Camp #26,Sierra Nevada Brewing Co.,Märzen / Oktoberfest,3.78,"nice auburn impressions, tons of clarity, solid inch of off white head. aroma was a little bit sweet and nutty. taste gave a little more sweetness, stayed away from hops and bitterness, relatively light bodied. nothing almond came out of it that was obvious. kind of a fancied up oktoberfest ...","[nice, auburn, impressions, tons, clarity, solid, inch, white, head, aroma, little, bit, sweet, nutty, taste, gave, little, sweetness, stayed, away, hops, bitterness, relatively, light, bodied, nothing, almond, came, obvious, kind, fancied, oktoberfest, good, really, change, anything, use, almon..."
3,Perpetual Burn IPA,Rock Bottom Restaurant & Brewery,American IPA,3.88,"s - piney, a little cat piss in a good way, skunky bud. t - some caramel front, tons of hops at the finish. m - medium body, a little too sharp and unbalanced for me. d - very sessionable considering the hops and abv.","[, piney, little, cat, piss, good, way, skunky, bud, , caramel, front, tons, hops, finish, , medium, body, little, sharp, unbalanced, me, , sessionable, considering, hops, abv]"
4,Hop Monster DRIPA,Iron Hill Brewery & Restaurant,American Double / Imperial IPA,4.14,"double red ipa, full-bodied rich caramel malt character with a generous amount of american hops. beware the monster! a: this was a clear, amber tinted copper color. it was served to me with a firm, frothy cap. s: sugar cookie/short bread malts with a little honey and a very strong blast of tro...","[double, red, ipa, fullbodied, rich, caramel, malt, character, generous, amount, american, hops, beware, monster, a, clear, amber, tinted, copper, color, served, firm, frothy, cap, s, sugar, cookieshort, bread, malts, little, honey, strong, blast, tropical, fruit, hops, fresh, appealing, t, tons..."


In [45]:
# Do tf-idf transformation
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(df.iloc[:, 5].values)
tfidf_df = pd.DataFrame(tfidf.toarray())
# Fix column names
#tfidf_df = pd.concat([data.iloc[:, 0:4], tfidf_df], axis=1, ignore_index=True)
#tfidf_df.columns = cols

TypeError: no supported conversion for types: (dtype('O'),)

In [44]:
df.iloc[:,5]

0     [smell, , soft, hop, aroma, significant, malt, scents, one, smells, creamy, taste, , creamy, is, traditional, irish, flavors, come, tongue, creamy, like, cream, ale, close, malt, big, buttery, smooth, hops, unique, sharp, hop, flavor, easy, saturated, wellmixed, blend, plays, complimenting, seco...
1     [tap, dfh, rehoboth, collab, eatily, cardamom, red, wine, must, golden, orange, no, head, typical, dfh, yeast, aroma, some, spice, maybe, belgian, influence, sweet, spicy, somewhat, fruity, much, old, ale, characteristic, light, that, still, tasty, cardamom, add, nice, flavor, still, kind, lacki...
2     [nice, auburn, impressions, tons, clarity, solid, inch, white, head, aroma, little, bit, sweet, nutty, taste, gave, little, sweetness, stayed, away, hops, bitterness, relatively, light, bodied, nothing, almond, came, obvious, kind, fancied, oktoberfest, good, really, change, anything, use, almon...
3                                                                                 

In [42]:
df.head()

,name,brewery,style,rating,review,words
0,Big Rock Ale,Big Rock Brewery,Scottish Ale,3.90,"smell  soft hop aroma with significant malt scents. this one smells very creamy. taste  and creamy it is. the traditional irish flavors come out at the tongue. this is creamy, not like a cream ale, but close. the malt is big, buttery, and very smooth. the hops are very unique. this is not a sh...","[smell, , soft, hop, aroma, significant, malt, scents, one, smells, creamy, taste, , creamy, is, traditional, irish, flavors, come, tongue, creamy, like, cream, ale, close, malt, big, buttery, smooth, hops, unique, sharp, hop, flavor, easy, saturated, wellmixed, blend, plays, complimenting, seco..."
1,Flip Ale,Dogfish Head Craft Brewery,Old Ale,4.08,on tap at dfh rehoboth... collab with eatily... cardamom and red wine must. golden orange. .no head. typical dfh yeast aroma. ..some spice and maybe a belgian influence. sweet spicy and somewhat fruity.. not much old ale characteristic. too light for that. its still tasty. the cardamom does add...,"[tap, dfh, rehoboth, collab, eatily, cardamom, red, wine, must, golden, orange, no, head, typical, dfh, yeast, aroma, some, spice, maybe, belgian, influence, sweet, spicy, somewhat, fruity, much, old, ale, characteristic, light, that, still, tasty, cardamom, add, nice, flavor, still, kind, lacki..."
2,The Almond Marzen Project - Beer Camp #26,Sierra Nevada Brewing Co.,Märzen / Oktoberfest,3.78,"nice auburn impressions, tons of clarity, solid inch of off white head. aroma was a little bit sweet and nutty. taste gave a little more sweetness, stayed away from hops and bitterness, relatively light bodied. nothing almond came out of it that was obvious. kind of a fancied up oktoberfest ...","[nice, auburn, impressions, tons, clarity, solid, inch, white, head, aroma, little, bit, sweet, nutty, taste, gave, little, sweetness, stayed, away, hops, bitterness, relatively, light, bodied, nothing, almond, came, obvious, kind, fancied, oktoberfest, good, really, change, anything, use, almon..."
3,Perpetual Burn IPA,Rock Bottom Restaurant & Brewery,American IPA,3.88,"s - piney, a little cat piss in a good way, skunky bud. t - some caramel front, tons of hops at the finish. m - medium body, a little too sharp and unbalanced for me. d - very sessionable considering the hops and abv.","[, piney, little, cat, piss, good, way, skunky, bud, , caramel, front, tons, hops, finish, , medium, body, little, sharp, unbalanced, me, , sessionable, considering, hops, abv]"
4,Hop Monster DRIPA,Iron Hill Brewery & Restaurant,American Double / Imperial IPA,4.14,"double red ipa, full-bodied rich caramel malt character with a generous amount of american hops. beware the monster! a: this was a clear, amber tinted copper color. it was served to me with a firm, frothy cap. s: sugar cookie/short bread malts with a little honey and a very strong blast of tro...","[double, red, ipa, fullbodied, rich, caramel, malt, character, generous, amount, american, hops, beware, monster, a, clear, amber, tinted, copper, color, served, firm, frothy, cap, s, sugar, cookieshort, bread, malts, little, honey, strong, blast, tropical, fruit, hops, fresh, appealing, t, tons..."
